In [1]:
import pandas as pd
import re

In [2]:
file_indigo = r"/home/xena/Desktop/prateek/oneGo/onego_poc1/files/flight_schedules/FlightSchedule_Indigo.xlsx"

schedule_indigo = pd.read_excel(file_indigo)

In [3]:
schedule_indigo.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 107001 entries, 0 to 107000
Data columns (total 10 columns):
Origin               107001 non-null object
Destination          107001 non-null object
6E FlightNumber      107001 non-null object
Departure(LT)        107001 non-null object
Arrival(LT)          107001 non-null object
Flight Type          107001 non-null object
Via                  107001 non-null object
Days Of Operation    107001 non-null object
Effective From       107001 non-null object
Effective To         107001 non-null object
dtypes: object(10)
memory usage: 8.2+ MB


In [4]:
def separate_code(x):
    list = str(x).split('(')
    code = list[-1].rstrip(')').strip(' ')
    name = list[0].strip(' ')
#     print(name)
    return (name,code)

In [5]:
schedule_indigo.loc[:,'Origin_Code'] = schedule_indigo['Origin'].apply(lambda x:separate_code(x)[1])
schedule_indigo.loc[:,'Origin_Name'] = schedule_indigo['Origin'].apply(lambda x:separate_code(x)[0])

schedule_indigo.loc[:,'Destination_Code'] = schedule_indigo['Destination'].apply(lambda x:separate_code(x)[1])
schedule_indigo.loc[:,'Destination_Name'] = schedule_indigo['Destination'].apply(lambda x:separate_code(x)[0])

In [6]:
schedule_indigo.head()

,Origin,Destination,6E FlightNumber,Departure(LT),Arrival(LT),Flight Type,Via,Days Of Operation,Effective From,Effective To,Origin_Code,Origin_Name,Destination_Code,Destination_Name
0,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 321/6E 892,23:30,15:45,Connecting,BOM/CCU,Daily,12-Jul-19,25-Oct-19,AUH,Abu Dhabi,IXA,Agartala
1,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 6749/6E 892,23:30,15:45,Connecting,BOM/CCU,Daily,12-Jul-19,25-Oct-19,AUH,Abu Dhabi,IXA,Agartala
2,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 321/6E 374,23:30,18:50,Connecting,BOM/CCU,Except TUE,15-Jun-19,06-Aug-19,AUH,Abu Dhabi,IXA,Agartala
3,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 321/6E 276,23:30,17:40,Connecting,BOM/CCU,Except TUE,15-Jun-19,20-Jul-19,AUH,Abu Dhabi,IXA,Agartala
4,Abu Dhabi (AUH),Agartala (IXA),6E 1834/6E 321/6E 798,23:30,14:50,Connecting,BOM/CCU,Except TUE,15-Jun-19,25-Oct-19,AUH,Abu Dhabi,IXA,Agartala


In [7]:
schedule_indigo.rename(columns = {'Flight Type': 'Flight_Type'}, inplace =True)
direct = schedule_indigo[schedule_indigo.Flight_Type == 'Direct']

In [8]:
input_dir = "/home/xena/Desktop/prateek/oneGo/onego_poc1/files/" 
output_dir = "/home/xena/Desktop/prateek/oneGo/onego_poc1/files/otp_india/gtfs/" 
airports_file = input_dir + "airports.py"
selected_airports_file = input_dir + "selected_airports_50.txt"
selected_airports = set()

with open(selected_airports_file) as f:
    for line in f:
        selected_airports.add(line.split("\n")[0])

In [9]:
airports_dict = {}

with open(airports_file) as f:  # using this code takes care of closing the file
    for line in f:
        airport_key = line.split(' = ')[0]
        airport_details = line.split(' = ')[1]
        airports_dict[airport_key] = eval(airport_details)

In [10]:
indian_airport_dict = {}
for value in airports_dict.values():
    if value.get('country') == 'India':
        indian_airport_dict[value.get('code')] = value 

In [11]:
indian_airport_dict

{'AGR': {'city': 'Agra',
  'code': 'AGR',
  'worldareacode': '733',
  'country': 'India',
  'longitude': '77.96',
  'latitude': '27.15',
  'airportname': 'Kheria',
  'gmt': '+5.5'},
 'AGX': {'city': 'Agatti Island',
  'code': 'AGX',
  'worldareacode': '733',
  'country': 'India',
  'longitude': '72.2',
  'latitude': '10.83',
  'airportname': 'Agatti Island',
  'gmt': '+5.5'},
 'AJL': {'city': 'Aizawl',
  'code': 'AJL',
  'worldareacode': '733',
  'country': 'India',
  'longitude': '92.71',
  'latitude': '23.73',
  'airportname': 'Aizawl',
  'gmt': '+5.5'},
 'AKD': {'city': 'Akola',
  'code': 'AKD',
  'worldareacode': '733',
  'country': 'India',
  'longitude': '77.05',
  'latitude': '20.68',
  'airportname': 'Akola',
  'gmt': '+5.5'},
 'AMD': {'city': 'Ahmedabad',
  'code': 'AMD',
  'worldareacode': '733',
  'country': 'India',
  'longitude': '72.63',
  'latitude': '23.06',
  'airportname': 'Ahmedabad',
  'gmt': '+5.5'},
 'ATQ': {'city': 'Amritsar',
  'code': 'ATQ',
  'worldareacode': 

In [12]:
rows = []
for airport in selected_airports:
    stop_id = airport
    stop_code = airport
    stop_name = "{} airport".format(indian_airport_dict[airport].get('city'))
    stop_desc = stop_name
    stop_lat = indian_airport_dict[airport].get('latitude')
    stop_lon = indian_airport_dict[airport].get('longitude')
    zone_id = ""
    stop_url = ""
    location_type = 1 # https://gtfs.org/best-practices/#stopstxt
    parent_station = ""
    direction = ""
    position = ""
    row = [stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,direction,position]
    rows.append(row)



In [13]:
stops = pd.DataFrame(rows, columns=["stop_id","stop_code","stop_name","stop_desc","stop_lat","stop_lon","zone_id","stop_url","location_type","parent_station","direction","position"])
# stops = pd.DataFrame(rows)

In [14]:
stops.head()

,stop_id,stop_code,stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,direction,position
0,RJA,RJA,Rajahmundry airport,Rajahmundry airport,16.98,81.78,,,1,,,
1,GAU,GAU,Guwahati airport,Guwahati airport,26.1,91.58,,,1,,,
2,IXB,IXB,Siliguri airport,Siliguri airport,26.73,88.33,,,1,,,
3,IMF,IMF,Imphal airport,Imphal airport,24.86,93.93,,,1,,,
4,STV,STV,Surat airport,Surat airport,21.1,72.73,,,1,,,


In [15]:
stops_file = output_dir + "stops.txt"
stops.to_csv(stops_file, index = False)

In [16]:
routes_orig = pd.read_csv("/home/xena/Desktop/prateek/oneGo/onego_poc1/files/otp/gtfs_files/routes.txt", sep = ",")

In [17]:
trips_orig = pd.read_csv("/home/xena/Desktop/prateek/oneGo/onego_poc1/files/otp/gtfs_files/trips.txt", sep = ",")

In [21]:
stops_orig = pd.read_csv("/home/xena/Desktop/prateek/oneGo/onego_poc1/files/otp/gtfs_files/stops.txt", sep = ",")

In [24]:
stop_times_orig = pd.read_csv("/home/xena/Desktop/prateek/oneGo/onego_poc1/files/otp/gtfs_files/stop_times.txt", sep = ",")

In [23]:
stops_orig.head()

,stop_id,stop_code,stop_name,tts_stop_name,stop_desc,stop_lat,stop_lon,zone_id,stop_url,location_type,parent_station,direction,position
0,3,3.0,A Ave & Second St,ae avenue & second street,Eastbound stop in Lake Oswego (Stop ID 3),45.419388,-122.665197,B,http://trimet.org/#tracker/stop/3,0,NaN,East,Farside
1,6,6.0,A Ave & 8th St,ae avenue & 8th street,Eastbound stop in Lake Oswego (Stop ID 6),45.420217,-122.673070,B,http://trimet.org/#tracker/stop/6,0,NaN,East,Nearside
2,7,7.0,A Ave & 8th St,ae avenue & 8th street,Westbound stop in Lake Oswego (Stop ID 7),45.420411,-122.672680,B,http://trimet.org/#tracker/stop/7,0,NaN,West,Nearside
3,8,8.0,900 Block Abernethy,900 block abernethy,Eastbound stop in Oregon City (Stop ID 8),45.363509,-122.593886,B,http://trimet.org/#tracker/stop/8,0,NaN,East,At
4,9,9.0,Abernethy & Barclay,abernethy & barclay,Northbound stop in Gladstone (Stop ID 9),45.384645,-122.602873,B,http://trimet.org/#tracker/stop/9,0,NaN,North,Nearside


In [20]:
routes_orig.head()

,route_id,agency_id,route_short_name,route_long_name,route_type,route_url,route_color,route_text_color,route_sort_order
0,1,TRIMET,1.0,Vermont,3,http://trimet.org//schedules/r001.htm,NaN,NaN,400
1,2,TRIMET,2.0,Division,3,http://trimet.org//schedules/r002.htm,NaN,NaN,500
2,4,TRIMET,4.0,Fessenden,3,http://trimet.org//schedules/r004.htm,NaN,NaN,600
3,6,TRIMET,6.0,Martin Luther King Jr Blvd,3,http://trimet.org//schedules/r006.htm,NaN,NaN,800
4,8,TRIMET,8.0,Jackson Park/NE 15th,3,http://trimet.org//schedules/r008.htm,NaN,NaN,1000


In [19]:
trips_orig.head()

,route_id,service_id,trip_id,direction_id,block_id,shape_id,trip_type,wheelchair_accessible
0,1,W.555,9403907,0,169,428030,NaN,1
1,1,W.555,9403908,0,171,428030,NaN,1
2,1,W.555,9403909,0,167,428030,NaN,1
3,1,W.555,9403910,0,169,428030,NaN,1
4,1,W.555,9403911,0,171,428030,NaN,1


In [25]:
stop_times_orig.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,timepoint,continuous_drop_off,continuous_pickup
0,9403907,06:44:00,06:44:00,13170,1,45th Ave,0,0,0.0,1,NaN,NaN
1,9403907,06:45:26,06:45:26,7631,2,45th Ave,0,0,875.1,0,NaN,NaN
2,9403907,06:47:31,06:47:31,7625,3,45th Ave,0,0,2162.3,0,NaN,NaN
3,9403907,06:49:36,06:49:36,7612,4,45th Ave,0,0,3433.3,0,NaN,NaN
4,9403907,06:51:20,06:51:20,7616,5,45th Ave,0,0,4498.0,0,NaN,NaN
